## 1 - Sequence to Sequence Learning with Neural Networks

> Trong chuỗi bài viết này, chúng ta sẽ cùng xây dựng một machine learning model chuyển một sequence này sang sequence khác, sử dụng Pytorch và torchtext. Cụ thể trong notebook này, ta sẽ dịch từ tiếng Đức sang tiếng Anh. Tuy nhiên, models có thể áp dụng cho bất kỳ problem nào liên quan đến việc chuyển đổi sequence như: tổng hợp sequence (chuyển 1 sequence sang 1 sequence ngắn hơn trong cùng 1 ngôn ngữ).

> Ở notebook đầu tiên này, chúng ta sẽ cùng khởi động bằng việc hiểu các khái niệm chung thông qua việc implementing model từ paper [Sequence to Sequence Learning with Neural Networks](https://arxiv.org/pdf/1409.3215.pdf).

### Introduction

> Hầu hết các sequence-to-sequence (seq2seq) models là *encoder-decoder* models. *Encoder-decoder* models là các models sử dụng RNNs để encode source sentence (input) thành single vector (trong notebook này, single vector chính là *context vector*). Sau đó, vector này sẽ được decoded bởi RNNs thứ 2, RNNs này sẽ sinh ra mỗi từ một lần sau cho kết quả đầu ra tốt nhất có thể. 

![figure](./images/1.seq2seq_figure1.PNG)

> Ở trên là hình minh họa của bài toán dịch máy. Input (source sequence), "guten morgen" được truyền qua *embedding layer* (màu vàng) và được đưa vào encoder (màu xanh lá). Ở source sequence, ta chèn thêm ký tự đánh dấu đầu sequence (sos) và ký tự kết thúc sequence (eos). Mỗi một lần xử lý, input của encoder RNN sẽ là embedding của word hiện tại e($x_t$) và hidden state từ bước trước đó $h_{t-1}$. Encoder RNN sẽ trả về new hidden state $h_t$. Ta có thể tạm hiểu hidden state là biểu diễn vector của sentence. RNN có thể được biểu diễn là function của e($x_t$) và $h_{t-1}$: <br>
>>> $h_t$ = EncoderRNN(e($x_t$), $h_{t-1}$)

> Thuật ngữ RNN mang nghĩa tổng quan, nó có thể là bất cứ kiến trúc recurrent nào,như LSTM hoặc GRU.

> Ở đây, ta có $\bold{X}$ = {$x_1$, $x_2$, ..., $x_T$}, với $x_1$ = \<sos>, $x_2$ = guten, etc. Trạng thái ẩn ban đầu $\bold{h_0}$ thường được khởi tạo bằng 0 hoặc khởi tạo thông qua các tham số đã được train từ trước.

> Cuối cùng, $x_T$ được truyền vào RNN sau khi đi qua một *embedding layer*, sau đó ta sử dụng final hidden state $\bold{h_T}$ làm context vector. Context vector này chính là biểu diễn dưới dạng vector của toàn bộ source sentence (z = $\bold{h_T}$).

> Bây giờ, khi đã có context vector z, ta có thể bắt đầu decoding z thành output (target sentence), "good morning". Tương tự input, output cũng cần được chèn thêm các tokens đánh dấu bắt đầu và kết thúc sentence. Lúc này, đầu vào của decoder RNN sẽ là embedding $\bold{d}$ của từ hiện tại $\bold{d(y_t)}$ và hidden state từ bước phía trước $\bold{s_{t-1}}$. Hidden state ban đầu của decoder $\bold{s_0}$ sẽ là context vector của encoder $\bold{s_0}$ = z = $\bold{h_T}$. Tương tự như encoder, ta có thể biểu diễn decoder dưới dạng function như sau: <br>
>>> $\bold{s_t}$ = $\bold{DecoderRNN(d(y_t), s_{t-1})}$

> **NOTE**: Mặc dù input embedding layer: e và output embedding layer: d đều được thể hiện bằng màu vàng trên hình, nhưng chúng thực chất là 2 embedding layer khác nhau với các parameters khác nhau.

> Ở decoder, ta cần chuyển hidden state thành một từ, vì vậy mỗi khi ta sử dụng $s_t$ để dự đoán (bằng cách truyền nó qua một `Linear` layer, có màu tím), ta có biểu thức từ dự đoán $\bold{\hat{y}_t}$ như sau: <br>
>>> $\bold{\hat{y}_t}$ = $\bold{f(s_t)}$

> Các từ trong decoder luôn được sinh ra lần lượt, mỗi từ một lần. chúng ta luôn sử dụng \<sos> làm tokens đầu tiên trong decoder ($\bold{y_1}$), còn với các inputs phía sau ($\bold{y_{t>1}}$), ta sẽ sử dụng ground truth của từ tiếp theo ($\bold{y_t}$) nếu đang train và sử dụng giá trị dữ đoán của lớp phía trước ($\bold{\hat{y}_{t-1}}$). Đây được gọi là *teacher forcing*, đọc thêm tại [đây](https://machinelearningmastery.com/teacher-forcing-for-recurrent-neural-networks/).

> Trong quá trình training, chúng ta luôn biết trước số lượng từ trong target sentence, do đó ta sẽ dừng việc sinh từ khi đạt đủ số lượng. <Br>
> Trong giai đoạn inference, việc sinh từ được dừng khi model cho đầu ra là \<eos> sau một số lượng từ nhất định.

> Sau khi có được predicted target sentence, $\bold{\hat{Y}_t}$ = {$\bold{\hat{y}_1}$, $\bold{\hat{y}_2}$, ..., $\bold{\hat{y}_T}$} ta so sánh nó với giá trị thực $\bold{{Y}_t}$ = {$\bold{{y}_1}$, $\bold{{y}_2}$, ..., $\bold{{y}_T}$} để tính loss. Sau đó sử dụng loss đã tính được để update tất cả các parameters của model.

### Preparing Data

> Trong notebook này, ta sẽ sử dụng PyTorch để xây dựng model, sử dụng torchtext để xử lý data và spaCy để hỗ trợ việc xử lý data.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.legacy.datasets import Multi30k
from torchtext.legacy.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
import time

In [2]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

> Tiếp theo, ta sẽ tạo các tokenizers. Tokenizer được sử dụng để chuyển một sentence thành một list các tokens riêng biệt. <br>

> spacy có sẵn model để tokenize cho từng ngôn ngữ  ("de_core_news_sm" cho German và "en_core_web_sm" cho English). Các model này cần được load trước khi sử dụng.

In [3]:
spacy_de = spacy.load('de_core_news_sm')
spacy_en = spacy.load('en_core_web_sm')

> Tiếp theo, ta xây dựng các hàm tokenize. Các hàm này sẽ được truyền vào torchtext và thực hiện tokenize bên trong torchtext.

> Lưu ý là trong paper  [Sequence to Sequence Learning with Neural Networks](https://arxiv.org/pdf/1409.3215.pdf) việc đảo ngược sentence sẽ tạo sự phụ thuộc ngắn hạn, từ đó cho kết quả tốt hơn. Trong hàm tokenize German, ta thực hiện việc đảo ngược sentence.

In [4]:
def tokenize_de(text):
    """
    Tokenizes German text from a string into a list of strings (tokens) and reverses it
    """
    return [tok.text for tok in spacy_de.tokenizer(text)][::-1]

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings (tokens)
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

> Ta xử lý dữ liệu bằng torchtext.

In [5]:
SRC = Field(tokenize = tokenize_de, 
            init_token = '', 
            eos_token = '', 
            lower = True)

TRG = Field(tokenize = tokenize_en, 
            init_token = '', 
            eos_token = '', 
            lower = True)

> Tải dữ liệu [Multi30k dataset](https://github.com/multi30k/dataset). Dataset này chứa khoảng 30000 câu ở 3 thứ tiếng: English, German và French, mỗi câu khoảng 12 từ.

> Trương `exts` thông báo sử dụng những ngôn ngữ nào, trường `fields` xác định trường nào sử dụng ngôn ngữ nào.

In [6]:
train_data, valid_data, test_data = \
    Multi30k.splits(exts = ('.de', '.en'), 
                    fields = (SRC, TRG))        # SRC sẽ là tiếng Đức, TRG sẽ là tiếng Anh

> Kiểm ta số lượng examples:

In [7]:
print(f"Number of training examples: {len(train_data.examples)}")
print(f"Number of validation examples: {len(valid_data.examples)}")
print(f"Number of testing examples: {len(test_data.examples)}")

Number of training examples: 29000
Number of validation examples: 1014
Number of testing examples: 1000


In [8]:
print(vars(train_data.examples[0]))

{'src': ['.', 'büsche', 'vieler', 'nähe', 'der', 'in', 'freien', 'im', 'sind', 'männer', 'weiße', 'junge', 'zwei'], 'trg': ['two', 'young', ',', 'white', 'males', 'are', 'outside', 'near', 'many', 'bushes', '.']}


> Tiếp theo, ta thực hiện xây dựng *vocabulary* cho source và target language. Vocabulary là một look-up table, dùng để liên kết mỗi unique token với một index (index là một số nguyên dương). Vocabularies của source và target languages là khác nhau.

> Sử dụng đối số `min_freq` thông báo rằng ta sẽ chỉ sử dụng các tokens xuất hiện tối thiểu 2 lần trong toàn bộ văn bản để đưa vào vocab. Các tokens xuất hiện ít hơn 2 lần sẽ được xếp vào unknown token.

> Còn một lưu ý khác là vocabulary chỉ được xây dựng trên training set, không được xây dựng trên validation/test set. Việc này giúp ngăn chặn "information leakage".

In [9]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

In [10]:
print(f"Unique tokens in source (de) vocabulary: {len(SRC.vocab)}")
print(f"Unique tokens in target (en) vocabulary: {len(TRG.vocab)}")

Unique tokens in source (de) vocabulary: 7854
Unique tokens in target (en) vocabulary: 5892


In [11]:
TRG.vocab.itos[0:10]

['<unk>', '<pad>', '', 'a', '.', 'in', 'the', 'on', 'man', 'is']

> Bước cuối cùng trong xây dựng và xử lý dữ liệu là tạo iterator.

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device)

### Building the Seq2Seq Model

##### Encoder

> Theo paper nêu trên, encoder gồm 4 LSTM layers, tuy nhiên để giảm thời gian training, chúng ta giảm xuống còn 2 layers. Khái niệm về multi-layer RNNs giúp việc mở rộng từ 2 sang 4 layers khá dễ hiểu.

> Đối với multi-layer RNN, input sentence $\bold{X}$, sau khi được chuyển thành embedding sẽ đi vào layer đầu tiên của RNN. Hidden states của lớp đầu tiên $\bold{{H}}$ = {$\bold{{h}_1}$, $\bold{{h}_2}$, ..., $\bold{{h}_T}$} được sử dụng làm input của layer phía trên. Tương tự, hidden state output của layer bên dưới sẽ là input của layer ngay bên trên. Ta có biểu diễn hidden state của layer thứ nhất: <br>
>> $\bold{h^1_t}$ = $\bold{EncoderRNN^1(e(x_t), h^1_{t-1})}$

> Tương tự, hidden state của layer thứ hai: <br>
>> $\bold{h^2_t}$ = $\bold{EncoderRNN^2(h^1_t, h^2_{t-1})}$

> Sử dụng multi-layer RNN đồng nghĩa với việc ta cần khởi tạo hidden state cho mỗi layer: $h^l_0$, và mỗi layer sẽ cho một context vector $z^l$.

> Trong notebook này, chúng ta sẽ không đi chi tiết về LSTMs (nhấn vào [đây](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) để tìm hiểu sâu về LSTMs). Về cơ bản, ta chỉ cần biêt LSTMs là một dạng RNN, thay vì chỉ nhận input là hidden state và trả về hidden state, LSTMs nhận đầu vào là hidden state + cell state, đồng thời trả về hidden state + cell state.

>> $\bold{h_t}$ = $\bold{RNN(e(x_t), h_{t-1})}$

>> $\bold{(h_t, c_t)}$ = $\bold{LSTM(e(x_t), (h_{t-1}, c_{t-1}))}$

> Ta có thể tạm hiểu $c_t$ là một dạng hidden state khác. Tương tự như $h^l_0$, $c^l_0$ cũng được khởi tạo bằng tensor 0. Lúc này, context vector bao gồm final hidden state và final cell state: <br>
>> $z^l$ = ($h^l_T$, $c^l_T$)

> Tiếp theo, ta có biểu thức mở rộng cho multi-layer LSTMs: <br>
>> $\bold{(h^1_t, c^1_t)}$ = $\bold{EncoderLSTM^1(e(x_t), (h^1_{t-1}, c^1_{t-1}))}$ <br>
>> $\bold{(h^2_t, c^2_t)}$ = $\bold{EncoderLSTM^2(h^1_t, (h^2_{t-1}, c^2_{t-1}))}$

**NOTE:** chỉ có hidden state từ layer thứ nhất được truyền vào và làm input của layer thứ hai, cell state không được truyền đi. <br>

![figure](./images/1.seq2seq_figure2_lstm.PNG)

> Tiếp sau, chúng ta cùng xây dựng `Encoder` module. Class `Encoder` sẽ kế thừa `torch.nn.Module` và sử dụng  `super().__init__()` làm bản mẫu. Encoder nhận các đối số sau: <br>
>> * `input_dim` là kích thước của one-hot vectors được đưa vào encoder. Kích thước này bằng với vocabulary size.
>> * `emb_dim` là dimensionality của embedding layer. Layer này convert one-hot vectors thành các dense vector với số chiều là `emb_dim`.
>> * `hid_dim` là số chiều của hidden và cell states.
>> * `n_layers` là số lượng layers trong RNN (khi dùng multi-layer RNN).
>> * `dropout` là tỷ lệ dropout. Thực hiện dropout giúp trành việc overfitting. Đọc thêm và dropout tại [đây](https://www.coursera.org/lecture/deep-neural-network/understanding-dropout-YaGbR).

> Ở đây, chúng ta sẽ không đi chi tiết về embedding layer. Ta có thể hiểu cơ bản như sau: Embedding layer nhận đầu vào là địa chỉ (hay chỉ số) của word và chuyển nó thành dense vectors. Tham khảo [1](https://monkeylearn.com/blog/word-embeddings-transform-text-numbers/), [2](https://p.migdal.pl/2017/01/06/king-man-woman-queen-why.html), [3](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/) và [4](http://mccormickml.com/2017/01/11/word2vec-tutorial-part-2-negative-sampling/) để hiểu thêm về embedding.

> Ta thực hiện tạo embedding layer với `nn.Embedding`, tạo LSTM bằng `nn.LSTM` và dropout layer với `nn.Dropout`.

> Có một chú ý nhỏ ở đây là `dropout` argument trong LSTM xác định tỷ lệ dropout giữa các layer bên trong multi-layer RNN.

> Trong `forward` method, ta truyền vào source sentence $\bold{X}$, đã được converted thành dense vectors sau khi đi qua `embedding` layer, và được áp dụng dropout. Embedding này sẽ được truyền vào RNN. Do ta truyền toàn bộ sequence vào RNN, nó sẽ tự động tính toán các hidden states. Chú ý là ta cũng không truyền giá trị khởi tạo cho hidden hoặc cell state vào RNN. Lý do là nếu không truyền giá trị khởi tạo, RNN sẽ tự động khởi tạo bằng tensor 0.

> RNN trả về `outputs` (top-layer hidden state), `hidden` (final state cho mỗi layer $\bold{h_T}$, được xếp chồng lên sau mỗi layer) và `cell` (final cell state $\bold{c_T}$).

> Do ta chỉ quan tâm đến final hidden state và final cell states để tạo nên context vector, `forward` chỉ trả về `hidden` và `cell`.

> Kích thước của các tensor được comment trong code. Lưu ý là `n_directions` luôn bằng 1 (trong trường hợp này). Tuy nhiên với bidirectional RNNs, `n_directions` sẽ bằng 2.

In [13]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout=0.5) -> None:
        super().__init__()

        self.hid_dim = hid_dim
        self.n_layers = n_layers

        self.embedding = nn.Embedding(input_dim, emb_dim)

        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout=dropout)

        self.dropout = nn.Dropout(dropout)

    def forward(self, src):

        # src = [src len, batch size]

        embedded = self.dropout(self.embedding(src))

        # embedded = [src len, batch size, emb dim]

        outputs, (hidden, cell) = self.rnn(embedded)

        # outputs = [src len, batch size, hid dim * n directions]
        # hidden = [n layers * n directions, batch size, hid dim]
        # cell = [n layers * n directions, batch size, hid dim]

        # outputs are always from the top hidden layer
        
        return hidden, cell

In [14]:
for i, batch in enumerate(train_iterator):
    # print(batch.src[1])
    print(batch.trg[:, 0])
    a = batch.trg[:, 0]
    print(batch.trg.shape)
    break

tensor([  2,  13,  12,  89,  22,  31,   5, 242,   4,   2,   1,   1,   1,   1,
          1,   1,   1,   1,   1,   1,   1])
torch.Size([21, 128])


In [15]:
for i in a:
    print(TRG.vocab.itos[i], end=' ')

 woman with pink shirt sitting in chair .  <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> 

In [16]:
# test encoder
INPUT_DIM = len(SRC.vocab)
encoder = Encoder(INPUT_DIM, 256, 512, 2)
hidden, cell = encoder(batch.src)
print(hidden.shape)
print(cell.shape)

torch.Size([2, 128, 512])
torch.Size([2, 128, 512])


### Decoder

![figure](./images/1.seq2seq_figure3_decoder.PNG)

> `Decoder` class thực hiện từng bước decoding một, nó sẽ sinh ra từng token theo từng bước. Layer đầu tiện sẽ nhận hidden và cell state từ time-step trước, ($s^1_{t-1}, c^1_{t-1}$), và được đưa vào LSTM cùng với curent embedded token, $y_t$ để sinh ra new hidden và cel state, ($s^1_{t}, c^1_{t}$). Các layer con sẽ sử dụng hiden state từ lớp phía dưới $s^{l-1}_t$ và hidden state + cell state phía trước của layer hiện tại ($s^l_{t-1}, c^l_{t-1}$) để làm việc. Dưới đây là biểu thức của decoder:

>> $\bold{(s^1_t, c^1_t)}$ = $\bold{DecoderLSTM^1(d(y_t), (s^1_{t-1}, c^1_{t-1}))}$ <br>
>> $\bold{(s^2_t, c^2_t)}$ = $\bold{DecoderLSTM^2(s^1_t, (s^2_{t-1}, c^2_{t-1}))}$ <br>

> Lưu ý là giá trị khởi tạo cho hidden và cell states của decoder là context vector thu được từ encoder. ($s^l_0$, $c^l_0$) = $z^l$ = ($h^l_T$, $c^l_T$).

> Cuối cùng, ta truyền hidden state từ top layer của RNN, $s^L_t$ qua một linear layer, $\bold{f}$ để dự đoán token tiếp theo là gì: <br>
>> $\bold{\hat{y}_{t+1}}$ = $\bold{f(s^L_t)}$

> Các tham số và khởi tạo của decoder khá giống với `Encoder` class, ngoại trừ việc bây giờ chúng ta có `output_dim` là kích thước của vocabulary cho output/target. Ngoài ra, còn có thêm `Linear` layer phục vụ cho việc prediction từ top layer hidden state.

> bên trong `forward` method, ta có đầu vào sẽ là batch of input tokens, hidden states và cell states từ trước. Do ta thực hiện decoding một token 1 lúc, input tokens sẽ luôn có chiều dài là 1. Chúng ta cần sử dụng `unsqueeze` input tokens để thêm 1 dimension cho input tokens. 

> Tương tự như encoder, ta truyền data vao embedding layer và áp dụng dropout. Batch of embedded tokens này sẽ được truyền vào RNN cùng với previous hidden và cell states. RNN sẽ sinh ra `output` (hidden state từ top layer của RNN), new `hidden` state (mỗi layer có 1 hidden state, cộng dồn trên từng layer) và new `cell` state (mỗi layer có 1 cell state, cộng dồn trên từng layer). Tiếp theo, ta truyền `output` (sau khi loại bỏ sentence length dimension) vào linear layer để thu được `prediction`. Cuối cùng return `prediction`, new `hidden` state và new `cell` state.

**NOTE:** vì sequence length trong decoder luôn bằng 1, ta có thể sử dụng `nn.LSTMCell` thay cho `nn.LSTM` do `nn.LSTMCell` được thiết kế để xử lý batch of input không phải là sequence. `nn.LSTMCell` chỉ là một cell, còn `nn.LSTM` là gộp của nhiều cell. Sử dụng `nn.LSTMCell` trong trường hợp này, ta không con `unsqueeze` và thêm một dimension. Tuy nhiên, mỗi layer cần có một `nn.LSTMCell` trong decoder để đảm bảo `nn.LSTMCell` nhận được chính xác giá trị khởi tạo của hidden state từ encoder.

In [17]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout = 0.5) -> None:
        super().__init__()

        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers

        self.embedding = nn.Embedding(output_dim, emb_dim)

        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout=dropout)

        self.fc_out = nn.Linear(hid_dim, output_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, cell):

        # input = [batch size]
        # hidden = [n layers * n directions, batch size, hid dim]
        # cell = [n layers * n directions, batch size, hid dim]

        # n directions in the decoder will both always be 1, therefore:
        # hidden = [n layers, batch size, hid dim]
        # context = [n layers, batch size, hid dim]

        input = input.unsqueeze(0)

        # input = [1, batch size]

        embedded = self.dropout(self.embedding(input))

        # embedded = [1, batch size, emb dim]

        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))

        # output = [seq len, batch size, hid dim * n directions]
        # hidden = [n layers * n directions, batch size, hid dim]
        # cell = [n layers * n directions, batch size, hid dim]

        # seq len, n layers and n directions will always be 1 in the decoder, therefore:
        # output = [1, batch size, hid dim]
        # hidden = [n layers, batch size, hid dim]
        # cell = [n layers, batch size, hid dim]

        prediction = self.fc_out(output.squeeze(0))

        # prediction = [batch size, output dim]

        return prediction, hidden, cell

In [18]:
# test decoder
for i, batch in enumerate(train_iterator):
    break


INPUT_DIM = len(SRC.vocab)
encoder = Encoder(INPUT_DIM, 256, 512, 2)
hidden, cell = encoder(batch.src)
print(hidden.shape)
print(cell.shape)



OUTPUT_DIM = len(TRG.vocab)
decoder = Decoder(OUTPUT_DIM, 256, 512, 2)
output, hidden, cell = decoder(batch.trg[0, :], hidden, cell)
print(output.shape)
print(hidden.shape)
print(cell.shape)

torch.Size([2, 128, 512])
torch.Size([2, 128, 512])
torch.Size([128, 5892])
torch.Size([2, 128, 512])
torch.Size([2, 128, 512])


### Seq2Seq

> Cuối cùng, ta implement seq2seq model, thực hiện các công việc sau:
>> * đọc chuỗi đầu vào.
>> * sử dụng encoder để sinh ra context vectors.
>> * áp dụng decoder để sinh ra kết quả.

> Dưới đây là kiến trúc model Seq2Seq:

![seq2seq](./images/1.seq2seq_figure3_fullmodel.PNG)

> Class `Seq2Seq` nhận các đối số `Encoder`, `Decoder` và `device`.

> Khi xây dựng `Seq2Seq`, ta cần đảm bảo số lượng layer và số chiều của hidden state (và cả cell state nữa) ở `Encoder` và `Decoder` phải bằng nhau. Điều này không phải lúc nào cũng đúng. Đôi khi số lượng layers hoặc hidden dimension sizes trong seq2seq model không cần phải bằng nhau.

> `forward` method nhận đầu vào là source sentence, target sentence và tỷ lệ teacher-forcing. Tỷ lệ teacher-forcing được sử dụng khi training model. khi decoding, ở mỗi time-step, chúng ta sẽ dự đoán token tiếp theo trong target sequence từ token đã được decoded phía trước, $\hat{y}_{t+1}$ = f($s^L_t$)z. Với xác suất bằng với teaching forcing ratio (`teacher_forcing_ratio`) chúng ta sẽ sử dụng ground-truth của token tiếp theo trong sequence làm đầu vào của decoder tiếp theo. Tuy nhiên, với xác suất `1 - teacher_forcing_ratio`, ta sẽ sử dụng token mà model dự đoán ra làm đầu vào của decoder tiếp theo dù nó không khớp với giá trị thực của token tiếp theo.

> Trong `forward` method, đầu tiên chúng ta tạo một `outputs` tensor lưu trữ các giá trị mà ta dự đoán $\bold{\hat{Y}}$.

> Sau đó, ta nạp input/source sentence `src` vào encoder và thu được final hidden state + final cell states.

> First input của decoder là start of sequence (\<sos>) token. Do `trg` tensor đều được chèn thêm token (khi ta define `init_token` trong trường `TRG`) ta lấy $y_1$ bằng cách slicing đến nó. Chúng ta đều biết trước chiều dài của target sentences (`max_len`), ta lặp bấy nhiêu lần. Ký tự cuối cùng được đưa vào decoder là ký tự nằm **trước** token \<eos>. Token \<eos> không bao giờ được đưa vào deocoder.

> Trong quá trình lặp, ta thực hiện các công việc sau: <br>
>> * Truyền input, previous hidden và previous cell state ($y_t$, $s_{t-1}$ và $c_{t-1}$) vào decoder.
>> * Thu về giá trị dự đoán, next hidden và next cell state ($\hat{y}_{t+1}$, $s_t$, $c_t$) từ decoder.
>> * Lưu trữ giá trị prediction $\hat{y}_{t+1}$ (`output`) và tensor predictions $\hat{Y}$ (`outputs`).
>> * Kiểm tra điều kiện để thực hiện teacher force:
>>>> * Nếu đạt điều kiện thực hiện, `input` sẽ là giá trị ground-truth của next token trong sequence, $y_{t+1}$ (`trg[t]`).
>>>> * Nếu không đạt, `input` sẽ là giá trị được dự đoán của token tiếp theo $\hat{y}_{t+1}$ (`top1`). Ta lấy giá trị `top1` bằng `argmax` trên output tensor.

> Sau khi hoàn thành predictions, ta trả về tensor chứa tất cả các predictions, $\bold{\hat{Y}}$ (`outputs`).

**NOTE:** Vòng lặp của decoder bắt đầu từ 1 thay vì 0. Điều này có nghĩa là phần tử thứ 0 trong `outputs` luôn có giá trị 0. Dẫn đến `trg` và `outputs` sẽ trông như sau: <br>
>> trg = [,$y_1$,$y_2$,$y_3$,] <br>
>> outputs = [0,$\hat{y}_1$,$\hat{y}_2$,$\hat{y}_3$,]

Sau đó, để tính loss, ta loại bỏ phần tử thứ 0 ở mỗi tensor:

>> trg = [$y_1$,$y_2$,$y_3$,] <br>
>> outputs = [$\hat{y}_1$,$\hat{y}_2$,$\hat{y}_3$,]

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device) -> None:
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"

    def forward(self, src, trg, teacher_forcing_ratio = 0.5):

        # src = [src len, batch size]
        # trg = [trg len, batch size]
        # teacher_forcing_ratio is probability to use teacher forcing
        # e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time

        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim

        # tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)

        # last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)

        # first input to the decoder is the <sos> tokens
        input = trg[0,:]

        for t in range(1, trg_len):
                
            # insert input token embedding, previous hidden and previous cell states
            # receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)

            # place predictions in a tensor holding predictions for each token
            outputs[t] = output

            # decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio

            # get the highest predicted token from our predictions
            top1 = output.argmax(1) 

            # if teacher forcing, use actual next token as next input
            # if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

### Training the Seq2Seq Model

> Tiếp theo, chúng ta cùng khởi tạo model. Như đã đề cập từ trước, kích thước đầu vào `input_dim` và kích thước đầu ra `output_dim` bằng kích thước của source vocabulary và target vocabulary. Kích thước embedding và dropout cho encoder, decoder có thể khác nhau nhưng số layers và kích thước hidden/cell states phải giống nhau.

In [ ]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

> Tiếp theo, ta khởi tạo weights cho model. Ở đây ta khởi tạo weights theo phân phối chuẩn -0.08 đến +0.08. Ta tạo một hàm `init_weights` sau đó sử dụng `apply` để áp dụng cho model.

In [ ]:
def init_weights(model):
    for name, param in model.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)

model.apply(init_weights)

> Hàm tính số lượng tham số cần train của model.

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

> Định nghĩa optimizer.

In [ ]:
optimizer = optim.Adam(model.parameters())

> Định nghĩa loss function. Ở đây ta sử dụng hàm `CrossEntropyLoss`. Hàm loss này sẽ tính trung bình sai số trên mỗi token, tuy nhiên ta còn truyền thêm chỉ chố của padding token bằng `ignore_index` nên model sẽ không tính loss khi token đó là padding token.

In [ ]:
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

> Tiếp theo, ta xây dựng hàm huấn luyện model. Đầu tiên, ta cần dùng `model.train()` để đưa model vào trạng thái huấn luyện (bật dropout và batch normalization nếu có).

> Như đã đề cập từ trước, vòng lặp trong decoder bắt đầu từ chỉ số 1, không phải chỉ số 0. Điều này có nghĩa phần tử thứ 0 trong `outputs` luôn bằng 0: <br>
>> trg = [,$y_1$,$y_2$,$y_3$,] <br>
>> outputs = [0,$\hat{y}_1$,$\hat{y}_2$,$\hat{y}_3$,]

Sau đó, để tính loss, ta loại bỏ phần tử thứ 0 ở mỗi tensor:

>> trg = [$y_1$,$y_2$,$y_3$,] <br>
>> outputs = [$\hat{y}_1$,$\hat{y}_2$,$\hat{y}_3$,]

> Ở mỗi lần lặp, ta thực hiện các công việc sau:
>> * Lấy source và target sentences từ batch, X và Y.
>> * Zero gradients được tính từ batch trước.
>> * Truyền source và target và model để lấy output, $\hat{Y}$.
>> * Do loss function chỉ lam việc với 2d inputs và 1d targets nên ta cần kéo dãn chúng với `view`. (Đồng thời ta cũng xóa bỏ cột đầu tiên ở target và output như đã đề cập bên trên).
>> * Tính gradients với `loss.backward()`.
>> * Cắt gradients để ngăn chặn hiện tượng bùng nổ gradients (đây là 1 vấn đề phổ biến trong RNN).
>> * Cập nhật tham số.
>> * Tính tổng loss.

In [ ]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

> Evaluation loop khá giống với training loop. Dưới đây là một vài điểm điều chỉnh: <br>
>> * Đầu tiên ta cần bật chế độ evaluation (tắt dropout và batch normalization) trong model với `model.eval()`.
>> * Tiếp theo sử dụng `with torch.no_grad()` để tắt việc tính đạo hàm.
>> * Tắt teacher-forcing để dự đoán.

In [ ]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

> Hàm tính thời gian.

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

> Bắt đầu train model.

In [ ]:
# check folder exists, if not create it
import os
if not os.path.exists('models'):
    os.makedirs('models')

In [ ]:
N_EPOCHS = 5
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), './models/tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

> Load model và đánh giá.

In [ ]:
model.load_state_dict(torch.load('./models/tut1-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

## END